# Don't forget to change the paths of your object detection file, it's too big to put on git

In [1]:
# import the necessary packages
#840

from imutils.video import WebcamVideoStream
from imutils.video import FPS
import os
import imutils
from IPython.display import display
from cv2 import *
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import tensorflow as tf
import time
import os
import numpy as np
import matplotlib.image as mpimg

In [2]:
#!pip install tensorflow==1.15
#!pip list | grep tensorflow

In [3]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import visualization_utils as vis_util

In [4]:
#!pip install tensorflow==2.00
!pip list | grep tensorflow

tensorflow                             2.0.0      
tensorflow-estimator                   2.0.1      
tensorflow-object-detection-api        0.1.1      


In [5]:
path = "/Users/guilhermeviveiros/Desktop/models/research/object_detection"
MODEL_NAME = 'hand_graph'
PATH_TO_CKPT = path + '/' + MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join(path+'/training', 'labelmap.pbtxt')
NUM_CLASSES = 1

In [7]:
detection_graph = tf.compat.v1.get_default_graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.compat.v2.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [8]:
from object_detection.utils import label_map_util
label_map_util.tf = tf.compat.v1
tf.gfile = tf.io.gfile

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [9]:
def test_several_images(PATH_TO_TEST_IMAGES_DIR = "/Users/guilhermeviveiros/Desktop/Part_01/469"):
    
    TEST_IMAGE_PATHS = []

    for i in range(1,33):
        if(i < 10):
            path = os.path.join(PATH_TO_TEST_IMAGES_DIR, '0000{}.jpg'.format(i))
            TEST_IMAGE_PATHS.append(path)
        else:
            path = os.path.join(PATH_TO_TEST_IMAGES_DIR, '000{}.jpg'.format(i))
            TEST_IMAGE_PATHS.append(path)      
                            
    frames = []
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
            for image_path in TEST_IMAGE_PATHS:
               
                image = cv2.imread(image_path)
            
                image = imutils.resize(image, width=600)
                #image_np = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                image = cv2.Canny(image,100,200) 
                
               
                image_np_expanded = np.expand_dims(image_np,axis=0)
                image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                scores = detection_graph.get_tensor_by_name('detection_scores:0')
                classes = detection_graph.get_tensor_by_name('detection_classes:0')
                num_detections = detection_graph.get_tensor_by_name('num_detections:0')

                (boxes, scores, classes, num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: image_np_expanded})

                # Visualization of the results of a detection.
                vis_util.visualize_boxes_and_labels_on_image_array(
                          image_np,
                          np.squeeze(boxes),
                          np.squeeze(classes).astype(np.int32),
                          np.squeeze(scores),
                          category_index,
                          use_normalized_coordinates=True,
                          max_boxes_to_draw = 1,
                          min_score_thresh=0.7,
                          line_thickness=8)
            
                

                frames.append(image)
        
            save_frames(frames)

##if you wanna test the model in several images run the function bellow but change the path            
##test_several_images()

In [10]:
def save_frames(frames):
    
    for (step,frame) in enumerate(frames):
        
        if step < 10:
            imwrite("/Users/guilhermeviveiros/Desktop/LEI/Task1/Fotos/0" + str(step) + ".jpg", frame)
        else:
            imwrite("/Users/guilhermeviveiros/Desktop/LEI/Task1/Fotos/" + str(step) + ".jpg", frame)

# OPENCV WITHOUT THREADING FOR CAPTURING 
## grab a pointer to the video stream and initialize the FPS counter

In [11]:
# created a *threaded* video stream, allow the camera sensor to warmup,
# and start the FPS counter


print("[INFO] sampling THREADED frames from webcam...")
vs = cv2.VideoCapture(0)
i = 0


#with detection_graph.as_default():
#        with tf.Session(graph=detection_graph) as sess:
#g = tf.Graph()
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

with detection_graph.as_default():
    
    with tf.Session(graph=detection_graph) as sess:
        while True:
            
            ret,frame = vs.read()
            frame = cv2.flip(frame,1)
            frame = imutils.resize(frame, width=600)
            #frame = cv2.resize(frame, (400,400))
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
            image_np = frame
            #edges = cv2.Canny(frame,100,200) 
            image_np_expanded = np.expand_dims(image_np,axis=0)
    
            
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            
            (boxes, scores, classes, num_detections) = sess.run(
            [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
                    
                                    
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw = 2,
                min_score_thresh=0.9,
                line_thickness=8)
                    
            cv2.imshow("Frame", frame)
                    
                
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
        # do a bit of cleanup
        #vs.stream.release()
        vs.release()
        cv2.destroyAllWindows()
        

[INFO] sampling THREADED frames from webcam...
Instructions for updating:
non-resource variables are not supported in the long term


### Cunny Detection

In [17]:
# OpenCV program to perform Edge detection in real time 
# import libraries of python OpenCV  
# where its functionality resides 
import cv2  
  
# np is an alias pointing to numpy library 
import numpy as np 
  
  
# capture frames from a camera 
cap = cv2.VideoCapture(0) 
  
  
# loop runs if capturing has been initialized 
while(1): 
  
    # reads frames from a camera 
    ret, frame = cap.read() 
  
    # converting BGR to HSV 
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) 
      
    # define range of red color in HSV 
    lower_red = np.array([30,150,50]) 
    upper_red = np.array([255,255,180]) 
      
    # create a red HSV colour boundary and  
    # threshold HSV image 
    mask = cv2.inRange(hsv, lower_red, upper_red) 
  
    # Bitwise-AND mask and original image 
    res = cv2.bitwise_and(frame,frame, mask= mask) 
  
    # Display an original image 
    cv2.imshow('Original',frame) 
    frame = cv2.flip(frame,1)
    frame = imutils.resize(frame, width=600)
        
    # finds edges in the input image image and 
    # marks them in the output map edges 
    edges = cv2.Canny(frame,100,200) 
  
    # Display edges in a frame 
    cv2.imshow('Edges',edges) 
  
    # Wait for Esc key to stop 
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
                    
  
  
# Close the window 
cap.release() 
  
# De-allocate any associated memory usage 
cv2.destroyAllWindows()  


## Just an example of our final model

In [15]:
## import threading
import time
#def detect_hand():
    
import tensorflow.keras
from tensorflow.keras.models import load_model
print("[INFO] sampling THREADED frames from webcam...")
vs = cv2.VideoCapture(0)
detect_hand = True

frames = np.zeros(shape=(36,75,100,3))
i = 0

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

with detection_graph.as_default():
    
    with tf.Session(graph=detection_graph) as sess:
            
            model = load_model('../../simple_model/best_model_6_supersimple.h5')
            model._make_predict_function() 
            print("Ready to start")
            while True:
                
                ret,frame = vs.read()
                frame = cv2.flip(frame,1)
                frame = imutils.resize(frame, width=600)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                # check to see if the frame should be displayed to our screen
                if detect_hand == True:
                    
                    image_np = frame
                    image_np_expanded = np.expand_dims(image_np,axis=0)
                    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                    boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                    scores = detection_graph.get_tensor_by_name('detection_scores:0')
                    classes = detection_graph.get_tensor_by_name('detection_classes:0')
                    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
                      
                    
                        
                        
                    (boxes, scores, classes, num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: image_np_expanded})
                    
                    # Visualization of the results of a detection.
                    vis_util.visualize_boxes_and_labels_on_image_array(
                        image_np,
                        np.squeeze(boxes),
                        np.squeeze(classes).astype(np.int32),
                        np.squeeze(scores),
                        category_index,
                        use_normalized_coordinates=True,
                        max_boxes_to_draw = 2,
                        min_score_thresh=0.90,
                        line_thickness=8)
                    
                    cv2.imshow("Frame", frame)
                    
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break
                    
                    condition = scores[0][0]*100                    
                        
                    if(condition >= 90):
                        print(condition)
                        detect_hand = False
                    
                    #frames_to_see.append(frame)
                            
                            
                #começa a processar o gesto
                if(detect_hand == False):
                    
                    cv2.imshow("Frame", frame)
                    
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break
                    
                    
                    frame = cv2.resize(frame, (100,75))
                    frames[i] =  frame
                    i +=1;
                    
                    if(i == 36):
                        l = model.predict([[frames]])
                        l = np.argmax(l)
                        if(l == 0): print("Doing other things")
                        elif(l==1): print("Swipping down")
                        elif(l==2): print("Swipping left")
                        elif(l==3): print("Zooming")   
                        detect_hand = True
                        i=0
                        
                        time.sleep(1)
                        print("Ready to capute the hand")
                        
            # do a bit of cleanup
            #vs.stream.release()
            vs.release()
            cv2.destroyAllWindows()
        

[INFO] sampling THREADED frames from webcam...
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Ready to start
94.38413977622986
Swipping down
Ready to capute the hand
93.51779222488403
Swipping down
Ready to capute the hand
93.54673624038696
Zooming
Ready to capute the hand
93.21122765541077
Swipping down
Ready to capute the hand
98.78085851669312
Zooming
Ready to capute the hand
95.32358646392822
Zooming
Ready to capute the hand
92.37041473388672
Zooming
Ready to capute the hand
94.8197603225708
Zooming
Ready to capute the hand
99.80244040489197
Doing other things
Ready to capute the hand
97.39797115325928
Zooming
Ready to capute the hand
97.89173603057861
Doing other things
Ready to capute the hand
97.18940854072571
Zooming
Ready to capute the hand


KeyboardInterrupt: 

In [ ]:
## def predict(frames): 
    l = np.argmax(model.predict([[frames]]))  
    print(model.predict([[frames]]))
    if(l == 0): print("Doing other things")
    elif(l==1): print("Swipping down")
    elif(l==2): print("Swipping left")
    elif(l==3): print("Zooming")   

In [108]:
predict(frames)                    

[[0. 1. 0. 0.]]
Swipping down
